In [ ]:
# prompt: write code for my streamlit app for this model

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib # To save and load the trained model

st.title('Crop Yield Impact Prediction')

st.write("""
This app predicts the yield impact for Crop 1 based on weather and crop data.
""")

# Function to load data
@st.cache_data
def load_data():
    # In a real Streamlit app, you would typically have the data file available
    # either by uploading it or having it in the same directory as the app.
    # For this example, let's assume 'Nigeria_Weather_Crop_Data_Combined.csv' is available.
    try:
        df = pd.read_csv('Nigeria_Weather_Crop_Data_Combined.csv')
        return df
    except FileNotFoundError:
        st.error("Data file 'Nigeria_Weather_Crop_Data_Combined.csv' not found. Please make sure the file is in the same directory as the app.")
        return None

df = load_data()

if df is not None:
    st.subheader('Raw Data')
    st.dataframe(df.head())

    # Define features (x) and target variable (y)
    # Assuming 'Yield Impact Crop 1 (%)' is the target column
    target_column = 'Yield Impact Crop 1 (%)'
    if target_column not in df.columns:
        st.error(f"Target column '{target_column}' not found in the dataset.")
    else:
        x = df.drop(['Yield Impact Crop 1 (%)','Yield Impact Crop 2 (%)','Yield Impact Crop 3 (%)'], axis=1, errors='ignore')
        y = df[target_column]

        # Split data
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

        # Identify categorical columns
        categorical_features = x_train.select_dtypes(include=['object']).columns

        # Create a column transformer
        preprocessor = ColumnTransformer(
            transformers=[
                ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)
            ],
            remainder='passthrough'
        )

        # Create and train the pipeline
        model = Pipeline(steps=[('preprocessor', preprocessor),
                                ('regressor', LinearRegression())])

        st.subheader('Training Model')
        with st.spinner('Training the model...'):
            model.fit(x_train, y_train)
        st.success('Model training complete!')

        # Make predictions
        y_pred = model.predict(x_test)

        # Evaluate the model
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        st.subheader('Model Evaluation')
        st.write(f"Mean Squared Error: {mse:.2f}")
        st.write(f"R-squared: {r2:.2f}")

        # Plot actual vs. predicted
        st.subheader('Actual vs. Predicted Yield Impact')
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.scatter(y_test, y_pred, alpha=0.5)
        ax.set_xlabel(f"Actual {target_column}")
        ax.set_ylabel(f"Predicted {target_column}")
        ax.set_title(f"Actual vs. Predicted {target_column}")
        ax.grid(True)
        st.pyplot(fig)

        # --- Prediction Section ---
        st.subheader('Make a Prediction')
        st.write("Enter the features to get a yield impact prediction.")

        # Create input fields for features
        # This part needs to be customized based on your actual features in x
        # For demonstration, let's create some generic inputs.
        # You should replace this with inputs corresponding to your 'x' columns.
        input_data = {}
        for col in x_train.columns:
            dtype = x_train[col].dtype
            if dtype == 'object':
                unique_values = df[col].unique().tolist()
                input_data[col] = st.selectbox(f"Select {col}", unique_values)
            elif dtype in ['int64', 'float64']:
                 # Add a check if the column is one of the ignored target columns
                if col not in ['Yield Impact Crop 2 (%)', 'Yield Impact Crop 3 (%)']:
                    # Use text_input first and convert to appropriate type
                    user_input = st.text_input(f"Enter value for {col}", value='0')
                    try:
                        if dtype == 'int64':
                             input_data[col] = int(user_input)
                        else: # float64
                             input_data[col] = float(user_input)
                    except ValueError:
                        st.warning(f"Invalid input for {col}. Please enter a number.")
                        input_data[col] = 0 # Default or handle error

        # Create a DataFrame from the input data
        input_df = pd.DataFrame([input_data])

        # Ensure the input DataFrame has the same columns and order as the training data
        # This is crucial for the preprocessor and model.
        # We need to add any missing columns from the training data with default values (e.g., 0)
        # and ensure the order is correct.
        missing_cols = set(x_train.columns) - set(input_df.columns)
        for c in missing_cols:
            input_df[c] = 0 # You might need a more sophisticated default based on your data

        # Reorder columns to match the training set
        input_df = input_df[x_train.columns]


        if st.button('Predict'):
            try:
                prediction = model.predict(input_df)
                st.subheader('Prediction Result')
                st.write(f"The predicted Yield Impact for Crop 1 is: {prediction[0]:.2f}%")
            except Exception as e:
                st.error(f"An error occurred during prediction: {e}")

